In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import pandas as pd
import re
import indicators as ind
import t_test as tt
nlp = en_core_web_sm.load()

# Experiment

- Source: don't care about count just presence
- Calculate Source % measure
  - precision and recall
- 1) differentiate sources conditioned on news site
- 2) correlate sources with type of piece (factual/opinionated/conspiracy) 
  - look any variations such as article size, percentage of sourcing, ect.
- 3) graph relationships (sentiment vs (number of sources/type of sources/etc)
  - other visualizations
  - media site level vs article level

  




In [2]:
# Load English tokenizer, tagger,  
# parser, NER and word vectors 
# nlp = spacy.load("en_core_web_sm") 

In [2]:
# code

indicators = ['said', 'added', 'told', 'blasted', 'suggested', 
                         'claimed', 'called', 'conceded', 'argued', 'announced', 
             'noted', 'shouted', 'testified', 'rejected']

special_indicators = ['according', 'According' ]
special_dict = {'according': 'to', 'According': 'to'}

# get rid of this logic, not needed
exclude = ['Fox', 'News']

# replace with regex # R-  D- 
rep = ['R-Ariz.', 'R-Maine', 'R-Alaska', 'R-Ky.', 'R-S.C.', 'R-S.D.' ]

def replace_words(article):
    regex = r'(,\s)?[R|D]{1}\-[^\s|^,]+'
    article = re.sub(regex, '', article)
    article = article.replace('!', '.').replace("?", '.').split('.')
    return article


def collect_sources(article, indicators=ind.indicators):
    # may need to add some more cases to handle splitting setences correctly
    wl =3  # window_length
    article = article.replace('Sen.', 'Sen').replace('U.S.', 'US').replace('\n','')
    # split into sentences based on .|?|!
    article = replace_words(article)
    df = pd.DataFrame(columns=['source', 'type', 'indicator', 'quoutes', 'sentence'])
    for sent in article:
        sentence = nlp(sent)
        m_sent = [(str(token), str(token.ent_type_), str(token.pos_)) for token in sentence]
        # vars to construct df
        source = ''
        tipe = ''
        ind = ''
        quote_bool = 0
        # loop through modified sentences, constructing source data frame
        # add tweets
        tweet = sent.find('(@')
        if tweet != -1:
            manipulate = sent[tweet::].split(')')[0][1::]
            df.loc[len(df)] = [manipulate, 'twitter', '', '', sent]
        for idx, tup in enumerate(m_sent):
            if tup[0] in indicators:
                ind = tup[0]
#                 print(m_sent[max(idx-wl, 0): idx])
                # before
                for j in m_sent[max(idx-wl, 0): idx]:
                    if (j[1] in {'NORP', 'ORG', 'PERSON', ''} and (j[2]=='PROPN' or j[2]=='NOUN')
                        and j[0] not in exclude) or j[0] == 'source' or j[0].lower()=='he' or j[0].lower()=='she':
                        source = source + j[0] + ' '
                        tipe = j[1]
                    # quotes need to be near indicator
                    if j[0] in ['“' , '”' , '"']:
                            quote_bool = 1
                        
                # after
                if source == '':
                    for j in m_sent[idx: min(idx+wl, len(m_sent))]:
                        if (j[1] in {'NORP', 'ORG', 'PERSON', ''} and j[2]=='PROPN' 
                            and j[0] not in exclude) or j[0] == 'source' or j[0].lower()=='he' or j[0].lower()=='she':
                            source = source + j[0] + ' '
                            tipe = j[1]
                        # quotes need to be near indicator
                        if j[0] in ['“' , '”' , '"']:
                                quote_bool = 1
                                
            # special indicators               
            elif tup[0] in special_indicators:
                ind = tup[0] + ' ' + special_dict[tup[0]]
                indice = sent.find(ind)
                if indice != -1:
                    source = sent[indice::].replace(ind, '').strip()
                    source = ' '.join(source.split(' ')[0:2])
                
            # write row
            if ind!='' and source!='':
                df.loc[len(df)] = [source, tipe, ind, quote_bool, sent]
                # reset vars
                source = ''
                tipe = ''
                ind = ''
                quote_bool = 0
    return df


def get_count(df):
    # get count on last part of source name
    df['source'] = df['source'].apply(lambda x: x.split(' ')[-2])
    return df['source'].value_counts()


def check_coverage(result_df, result_list):
    coverage_total = len(result_list)
    so = list(set(result_df['source']))
    sources = [s.strip().split(' ')[-1].lower() for s in so]
    counter = 0
    uncounted = 0
    counted = []
    additionals = []
    for r in result_list:
        res = r.split(' ')[-1].lower()
        if res in sources:
            counter +=1
            counted.append(r)
        else:
            uncounted +=1
    # print report
    
    print('Coverage:', counter/coverage_total)
    print('================================')
    print('Expected', result_list)
    print('------------------------------')
    print('Seen & Counted', counted)
    print('------------------------------')
    print('Misses', list(set(result_list).difference(set(counted))) )  
    print('------------------------------')
    print('Seen & Not Counted', set(so).difference(set(counted)) )
    print('------------------------------')

# Test Case

In [25]:
ex_n = ind.articleFoxNews2
zn = collect_sources(ex_n)

In [26]:
check_coverage(zn, ind.resultFoxNews2)

Coverage: 1.0
Expected ['two officials', 'John N. Kennedy', '@FoxNewsSunday', 'Booker']
------------------------------
Seen & Counted ['two officials', 'John N. Kennedy', '@FoxNewsSunday', 'Booker']
------------------------------
Misses []
------------------------------
Seen & Not Counted {'he ', 'He ', 'Republican ', 'Booker ', 'Sen Dianne ', '@SenJohnKennedy', 'Kennedy '}
------------------------------


In [23]:
zn

,source,type,indicator,quoutes,sentence
0,two officials,,According to,0,According to two officials who spoke anonymou...
1,Kennedy,PERSON,said,0,"Kennedy said on ""Fox News Sunday"" that the Ka..."
2,Republican,NORP,told,0,"""""I'm fairly confident that our Founding Fathe..."
3,Kennedy,PERSON,said,0,"Kennedy said Chairman Charles Grassley ( ""did ..."
4,Kennedy,PERSON,said,1,"""The only thing missing was the genitalia-shap..."
5,@SenJohnKennedy,twitter,,,co/MBOgroR0Sf — John Kennedy (@SenJohnKennedy)...
6,Sen Dianne,ORG,blasted,0,co/MBOgroR0Sf — John Kennedy (@SenJohnKennedy)...
7,He,,said,0,He also said the situation is one where many ...
8,he,,said,1,are looking at this and they're thinking that...
9,@FoxNewsSunday,twitter,,,co/lprtTBSKCk — FoxNewsSunday (@FoxNewsSunday)...


SyntaxError: invalid syntax (<ipython-input-6-6240dde41dae>, line 1)

In [4]:

znn = collect_sources(tt.article_1004)

In [5]:
znn

,source,type,indicator,quoutes,sentence
0,@FoxNews,twitter,,,com/du6idCXrUF — Fox News (@FoxNews) September...
1,Martha MacCallum,PERSON,reported,0,com/du6idCXrUF — Fox News (@FoxNews) September...
2,Fitzgerald,PERSON,said,0,"Fitzgerald said she was ""absolutely shocked"" a..."
3,Kavanaugh,ORG,said,0,"Fitzgerald said she was ""absolutely shocked"" a..."
4,Political Hatchet'Kane,PERSON,said,0,'Shame on Her': Bruce Calls Out Feinstein for ...
5,MacCallum,ORG,said,0,MacCallum said the situation reminds many peop...
6,Fitzgerald,PERSON,said,1,"""All of the years that I've known Brett, even ..."
7,allegations,,said,0,"""All of the years that I've known Brett, even ..."
8,Kane,ORG,said,0,Kane said that she drank socially with Kavanau...
9,she,,said,1,"""He never became 'someone different' after dri..."


In [6]:
check_coverage(znn, tt.result_1004)

Coverage: 1.0
Expected ['@FoxNews', 'Fitzgerald', 'Kane', 'MacCallum']
------------------------------
Seen & Counted ['@FoxNews', 'Fitzgerald', 'Kane', 'MacCallum']
------------------------------
Misses []
------------------------------
Seen & Not Counted {"Political Hatchet'Kane ", 'she ', 'Fitzgerald ', 'Kane ', 'Martha MacCallum ', 'Kavanaugh ', 'MacCallum ', 'allegations '}
------------------------------


In [2]:
example = 'Democratic Sen. Mazie Hirono blasted “the men in this country,” blaming them for sexual assault and urged them to “just shut up and step up” amid allegations of misconduct against President Trump’s Supreme Court nominee Judge Brett Kavanaugh.\n\nAt a press conference Tuesday evening, the Hawaii Democrat made the sweeping generalization against “the men” in the U.S. for “perpetuating” sexual assault.\n\n“I expect the men in this country and the men on this committee, because we all signed onto this letter, to demand an FBI investigation,” Hirono said, referring to a request that the FBI investigate the claims made by Kavanaugh’s accuser, Christine Blasey Ford.\n\nTRUMP SAYS \'WE\'LL HAVE TO MAKE A DECISION\' ON KAVANAUGH\'S FUTURE IF ACCUSER GIVES \'CREDIBLE\' TESTIMONY\n\n“But really, guess who is perpetuating all of these kinds of actions? It’s the men in this country. And I just want to say to the men in this country: just shut up and step up, do the right thing for a change,” Hirono said.\n\n“Not only do women, like Dr. Ford, who bravely comes forward, need to be heard, but they need to be believed,” Hirono added. “We cannot continue the victimization and the smearing of someone like Dr. Ford.”\n\nFord’s identity was revealed in a Washington Post article published over the weekend, following an anonymous letter obtained by Sen. Dianne Feinstein, D-Calif., alleging sexual assault. The California-based professor said that Kavanaugh, 36 years ago while in high school, pinned her down, tried to remove her bathing suit and put his hand over her mouth when she attempted to scream. Ford alleged that she was in the room with Kavanaugh and another man, who has been identified as Mark Judge.\n\nBoth Kavanaugh and Judge denied the allegations.\n\n“This is a completely false allegation. I have never done anything like what the accuser describes—to her or to anyone,” Kavanaugh said in a statement Monday. “Because this never happened, I had no idea who was making the accusation until she identified herself [Sunday.]”\n\nELIZABETH WARREN SLAMMED OVER EDITING OF KAVANAUGH VIDEO\n\nThe Senate Judiciary Committee invited both Ford and Kavanaugh to testify in either a public or private setting over the allegations.\n\nKavanaugh said he was “willing” to speak to the committee “in any way” deemed appropriate to “refute this false allegation” and to defend his “integrity.”\n\nFord’s attorney Monday suggested she would be willing to testify before the committee under oath, but has yet to officially accept the invitation to appear before the panel.\n\nFord has said she doesn’t want to appear until after an FBI investigation takes place.\n\nA highly-placed law enforcement source told Fox News on Wednesday that there will not be an FBI investigation into the allegations leveled at Kavanaugh.\n\nThe source told Fox News there cannot be a criminal investigation, because there is no crime alleged where the statute of limitations hasn\'t run. The source also said there are no allegations of a federal crime.\n\nKAVANAUGH ACCUSER CHRISTINE BLASEY FORD DEMANDS \'FULL INVESTIGATION\' BY FBI BEFORE TESTIFYING, IN LETTER FROM HER LAWYERS\n\nThe FBI only handles background check investigations, the source said, and should they find information, they would pass it along.\n\n"This is a political issue, not a law enforcement one," the source told Fox News.\n\nThe committee has already delayed Kavanaugh\'s confirmation vote, which was originally scheduled for Thursday. A hearing is now scheduled for Monday.\n\nSenate Judiciary Committee Chairman Chuck Grassley, R-Iowa, said that the hearing to examine the allegations set for Monday could be canceled if Ford does not accept the committee\'s invitation. Grassley said that his office has reached out several times to Ford and her attorneys to discuss her allegations, but has not received a response.\n\nFox News\' John Roberts contributed to this report.'

In [5]:

z = collect_sources(example)

In [6]:
z

,source,type,indicator,quoutes,sentence
0,Sen Mazie Hirono,ORG,blasted,0,Democratic Sen Mazie Hirono blasted “the men i...
1,assault,,urged,0,Democratic Sen Mazie Hirono blasted “the men i...
2,Hirono,ORG,said,1,“I expect the men in this country and the men ...
3,Hirono,ORG,said,1,And I just want to say to the men in this cou...
4,Hirono,ORG,added,1,"Ford, who bravely comes forward, need to be h..."
5,professor,,said,0,The California-based professor said that Kava...
6,Ford,ORG,alleged,0,Ford alleged that she was in the room with Ka...
7,Kavanaugh,PERSON,said,1,I have never done anything like what the accu...
8,Kavanaugh,,said,0,Kavanaugh said he was “willing” to speak to th...
9,’s attorney,,suggested,0,”Ford’s attorney Monday suggested she would be...


In [181]:
z['sentence'][2]

'\n\n“I expect the men in this country and the men on this committee, because we all signed onto this letter, to demand an FBI investigation,” Hirono said, referring to a request that the FBI investigate the claims made by Kavanaugh’s accuser, Christine Blasey Ford'

In [7]:
get_count(z)

source       5
Kavanaugh    4
Hirono       3
’s           1
Democrat     1
Grassley     1
Ford         1
Name: source, dtype: int64

In [66]:
ex2 = 'Senators nervously awaited the arrival of a new FBI report on sexual allegations that could make or break Brett Kavanaugh’s tottering Supreme Court nomination Wednesday as aggressive protesters and an unusually strong security response added to a feeling of high anxiety inside the U.S. Capitol.\n\nAs lawmakers anticipated the report, expected as early as Wednesday evening, three moderate GOP senators who could decide the conservative jurist’s fate rebuked President Donald Trump for mocking one accuser, Christine Blasey Ford, by mimicking her responses to questions at last week’s dramatic Senate Judiciary Committee hearing.\n\nTheir reactions left Republicans concerned that Trump had complicated their effort to cement Kavanaugh’s support in a chamber where the GOP holds a razor-thin 51-49 majority. Depending on when the FBI report arrived, Senate Majority Leader Mitch McConnell was expected to trigger a process that could lead to a crucial initial vote Friday and a climactic confirmation roll call over the weekend.\n\nInside the Capitol, mounting political strains over the approaching election-season showdown were mirrored by growing anxieties over senators’ security following frequent and at times aggressive demonstrations by anti-Kavanaugh protesters. Unusually large numbers of Capitol Hill Police officers restricted movements in corridors and formed wedges around senators walking through hallways. Some lawmakers also complained of being confronted outside their homes.\n\nOn the Senate floor, McConnell, R-Ky., claimed the protesters were “part of the organized effort” to derail Kavanaugh’s nomination and said, “There is no chance in the world that they’re going to scare us out of doing our duty.”\n\nSen. Susan Collins, R-Maine, told reporters that Trump’s Tuesday night lampooning of Ford at a Mississippi campaign rally was “just plain wrong.” Sen. Lisa Murkowski, R-Alaska, called it “wholly inappropriate and in my view unacceptable,” and Sen. Jeff Flake, R-Ariz., said on NBC’s “Today” show that the remarks were “kind of appalling.”\n\nThose GOP senators, along with Democrats Heidi Heitkamp of North Dakota and Joe Manchin of West Virginia, have yet to declare how they will vote on Kavanaugh. Other Republicans conceded that Trump’s insults could be damaging.\n\n“All of us need to keep in mind there’s a few people that are on the fence right now. And right now, that’s sort of where our focus needs to be,” said Sen. Bob Corker of Tennessee, who has traded barbs with Trump and will retire at year’s end.\n\nEven Trump ally Lindsey Graham, R-S.C., said at an event hosted by The Atlantic magazine: “I would tell him, knock it off. You’re not helping,”\n\nSenate Minority Leader Chuck Schumer, D-N.Y., said Trump’s insults of Ford marked a “new low.”\n\nThe California psychology professor has testified that a drunken Kavanaugh sexually abused her in a locked room at a high school party in the 1980s and has said she believed he was trying to rape her. Kavanaugh has denied her assertions and those of two other women, who have accused him of other instances of sexual misconduct in the 1980s.\n\nFord’s attorney complained Wednesday that the FBI has not contacted her for this week’s interviews. And Democrats argued that the investigation has been insufficient, lacking interviews with her, with Kavanaugh and others who Kavanaugh’s accusers have said could have knowledge about the alleged incidents.\n\nLawmakers said that once the FBI report arrived, senators and a small number of top aides would be allowed to read it in a secure room in the Capitol complex. Republicans have said they are working under an agreement governing background checks dating from the Obama administration, under which investigations are confidential and closely held.\n\nSens. Corker and Mike Rounds, R-S.D., said senators were expecting reports that FBI agents compile on their interviews with subjects, perhaps accompanied by a cover letter. But background checks do not traditionally contain investigators’ conclusions about who they believe is credible.\n\nWashington has been awaiting completion of the investigation since last week, when Flake, Collins and Murkowski pressured a reluctant Trump and GOP leaders to order the FBI to renew its background check of the 53-year-old Kavanaugh.\n\nThe FBI interviewed several people, including three who Ford has said attended a 1982 high school gathering in suburban Maryland where she says Kavanaugh’s attack occurred, plus another Kavanaugh friend. The agency has also spoken to a second woman, Deborah Ramirez, who has claimed Kavanaugh exposed himself to her at a Yale party when both were freshmen.\n\nWhile some senators from both parties have said they’d like at least a summary of the findings to be released, Senate procedures call for such checks to be kept confidential and it’s unclear what will be released, other than through leaks.\n\n“None of that stuff’s public,” Judiciary panel Chairman Chuck Grassley, R-Iowa, told reporters. “If you want people to be candid when they talk to the FBI, you ain’t going to make that public.”\n\nIn an interview, No. 2 Senate Democratic leader Richard Durbin of Illinois said McConnell was “hell bent” on confirming Kavanaugh this week.\n\nDemocrats also demanded that the FBI privately brief the Senate about the investigation before the chamber votes. McConnell rejected that request in a letter Wednesday to Schumer, saying Democrats would use it to delay Kavanaugh’s confirmation.\n\n___\n\nAssociated Press writers Eric Tucker, Michael Balsamo, Catherine Lucey, Zeke Miller, Padmananda Rama, Matthew Daly, Mary Clare Jalonick and Kevin Freking contributed from Washington. Will Weissert contributed from Austin, Texas.'

In [67]:
y = collect_sources(ex2)

In [68]:
y

,source,type,indicator,quoutes,sentence
0,security response,,added,0,Senators nervously awaited the arrival of a ne...
1,McConnell,ORG,claimed,0,"On the Senate floor, McConnell, claimed the pr..."
2,nomination,,said,0,"On the Senate floor, McConnell, claimed the pr..."
3,Susan Collins,PERSON,told,0,"”Sen Susan Collins, told reporters that Trump’..."
4,Lisa Murkowski,PERSON,called,0,"” Sen Lisa Murkowski, called it “wholly inappr..."
5,Jeff Flake,PERSON,said,0,"” Sen Lisa Murkowski, called it “wholly inappr..."
6,Republicans,NORP,conceded,0,Other Republicans conceded that Trump’s insul...
7,Sen Bob,PERSON,said,1,"And right now, that’s sort of where our focus..."
8,Lindsey Graham,PERSON,said,0,"Even Trump ally Lindsey Graham, said at an eve..."
9,Chuck Schumer,PERSON,said,0,"You’re not helping,”Senate Minority Leader Ch..."


In [32]:
y['sentence'][11]

' 2 Senate Democratic leader Richard Durbin of Illinois said McConnell was “hell bent” on confirming Kavanaugh this week'

In [17]:
ex2

'Senators nervously awaited the arrival of a new FBI report on sexual allegations that could make or break Brett Kavanaugh’s tottering Supreme Court nomination Wednesday as aggressive protesters and an unusually strong security response added to a feeling of high anxiety inside the U.S. Capitol.\n\nAs lawmakers anticipated the report, expected as early as Wednesday evening, three moderate GOP senators who could decide the conservative jurist’s fate rebuked President Donald Trump for mocking one accuser, Christine Blasey Ford, by mimicking her responses to questions at last week’s dramatic Senate Judiciary Committee hearing.\n\nTheir reactions left Republicans concerned that Trump had complicated their effort to cement Kavanaugh’s support in a chamber where the GOP holds a razor-thin 51-49 majority. Depending on when the FBI report arrived, Senate Majority Leader Mitch McConnell was expected to trigger a process that could lead to a crucial initial vote Friday and a climactic confirmati

In [186]:
get_count(y)

’s             2
Maine          1
NBC            1
Kavanaugh      1
McConnell      1
Bob            1
Democrats      1
Republicans    1
Ford           1
Trump          1
Name: source, dtype: int64

In [69]:
ex3 = 'West Virginia Attorney General Patrick Morrisey went after Democratic Sen. Joe Manchin’s decision on Friday to vote to confirm Judge Brett Kavanaugh to the U.S. Supreme Court.\n\nMorrisey called Manchin’s announcement a “craven political calculation” to salvage his political career. Manchin announced his support for Kavanaugh, President Donald Trump’s high court pick, minutes after Republican Maine Sen. Susan Collins said she would vote to confirm the judge.\n\n“Joe Manchin only votes in the interest of Joe Manchin,” Morrisey said in a statement issued Friday. “President Trump had all the votes he needed to confirm Judge Brett Kavanaugh, but Manchin waited – making a craven political calculation – in order to try to save his political career.”\n\n“When it matters most, Joe Manchin is with Hillary Clinton for President, Chuck Schumer and Obama for gun control, and is voting with Nancy Pelosi for amnesty for illegal immigrants,” Morrisey said.\n\nManchin voted to move Kavanaugh’s confirmation forward in a procedural vote on Friday. Manchin’s vote came after the West Virginia Democrat spent weeks on the sidelines regarding Kavanaugh’s confirmation.\n\nI will vote to support Supreme Court nominee Judge Brett Kavanaugh. pic.twitter.com/1FfuMTOZz8 — Senator Joe Manchin (@Sen_JoeManchin) October 5, 2018\n\n“With respect to any cases that may come before him impacting the 800,000 West Virginians with pre-existing conditions, Judge Kavanaugh assured me personally that he would consider the human impacts and approach any decision with surgical precision to avoid unintended consequences,” Manchin said in a statement.\n\n“That is why I voted to confirm Judge Kavanaugh’s nomination to serve on the Supreme Court because I believe he will rule in a manner that is consistent with our Constitution,” Manchin said. (RELATED: It’s The Final Chance We Have To Stop Global Warming … Until The Next Chance)\n\nManchin is leading Morrisey by an eight percentage point margin, according to a September poll. But Manchin is a Democrat in a red state where a vote against Trump’s Supreme Court nominee could have narrowed that comfortable lead.\n\nTrump nominated Kavanaugh, a federal judge, to replace retired Supreme Court Justice Anthony Kennedy in July, but late in the confirmation process a letter sent to California Sen. Dianne Feinstein’s office was leaked to the media.\n\nIn the letter, Christine Blasey Ford accused Kavaugh of sexual assault at a small high school party more than three decades ago. Another woman, Deborah Ramirez, accused Kavanaugh of exposing himself to her while drunk at a college dorm room party.\n\nKavanaugh denied the accusations, and a supplemental FBI background investigation was unable to corroborate accusations of sexual assault, according to the investigation’s summary.\n\nLiberal protesters gathered on Capitol Hill shouted “shame” at Manchin as he was interviewed about his vote.\n\nMorrisey also noted that the Senate Majority Political Action Committee would continue spending in support of Manchin even if he voted to confirm Kavanaugh, citing a Washington Examiner article.\n\n“West Virginians and President Trump can’t trust Joe Manchin to fight for our values because he is simply just another dishonest Washington liberal,” Morrisey said on Friday.\n\nFollow Michael on Facebook and Twitter\n\nContent created by The Daily Caller News Foundation is available without charge to any eligible news publisher that can provide a large audience. For licensing opportunities of our original content, please contact licensing@dailycallernewsfoundation.org.'

In [70]:
zz = collect_sources(ex3)

In [14]:
zz['sentence'][2]

' Manchin announced his support for Kavanaugh, President Donald Trump’s high court pick, minutes after Republican Maine Sen Susan Collins said she would vote to confirm the judge'

In [71]:
zz

,source,type,indicator,quoutes,sentence
0,Morrisey,,called,0,Morrisey called Manchin’s announcement a “crav...
1,Manchin,PERSON,announced,0,"Manchin announced his support for Kavanaugh, ..."
2,Sen Susan Collins,PERSON,said,0,"Manchin announced his support for Kavanaugh, ..."
3,Morrisey,ORG,said,1,“Joe Manchin only votes in the interest of Joe...
4,Morrisey,ORG,said,1,"”“When it matters most, Joe Manchin is with Hi..."
5,@Sen_JoeManchin,twitter,,,com/1FfuMTOZz8 — Senator Joe Manchin (@Sen_Joe...
6,Manchin,PERSON,said,1,com/1FfuMTOZz8 — Senator Joe Manchin (@Sen_Joe...
7,Manchin,PERSON,said,1,“That is why I voted to confirm Judge Kavanaug...
8,a September poll,,according to,0,(RELATED: It’s The Final Chance We Have To St...
9,the investigation’s summary,,according to,0,"Kavanaugh denied the accusations, and a supple..."


In [56]:
zz['sentence'][10]

'Liberal protesters gathered on Capitol Hill shouted “shame” at Manchin as he was interviewed about his vote'

In [170]:
t = 'Manchin is leading Morrisey by an eight percentage point margin, according to a September poll.'

In [173]:
idx = t.find('according to')
t = t[idx:-1]


In [174]:
t

'according to a September poll'

In [72]:
ex4 = 'Top Republicans voiced confidence Thursday that Brett Kavanaugh will be confirmed to the US Supreme Court this weekend, as they asserted that an FBI probe had found nothing to support sexual assault allegations against President Donald Trump\'s nominee.\n\n"Judge Kavanaugh should be confirmed on Saturday," Senator Chuck Grassley of Iowa, the chairman of the Senate Judiciary Committee, told reporters.\n\n"Hopefully, we\'re 48 hours away from having a new person on the Supreme Court," he added.\n\n"I hope we can just move forward and get this done," said another senior Republican senator, Orrin Hatch of Utah.\n\n"I apologize to him for the way he\'s been treated," Hatch said.\n\nHe defended the FBI investigation into the sexual assault accusations against the 53-year-old conservative appeals court judge, which Democrats have criticized as "incomplete" and "very limited."\n\n"We have found nothing, nothing to corroborate accusations against him," Hatch said. I\'m grateful to the FBI for their efforts in doing a thorough, very important investigation."\n\nKavanaugh\'s confirmation for a lifetime appointment on the nation\'s top court hinges on the votes of three Republicans, Jeff Flake of Arizona, Susan Collins of Maine and Lisa Murkowski, in the Senate, where Republicans hold a slim 51-49 majority.\n\nNone of the three was present at the press conference by the senior Republican lawmakers.\n\nAs they talked to reporters, a Democratic senator who had been seen as a potential vote for Kavanaugh, Heidi Heitkamp of North Dakota, said she would oppose his confirmation.\n\nThe White House expressed confidence that nothing in a supplemental FBI investigation prompted by sexual misconduct allegations against Kavanaugh would derail the Supreme Court nominee, as Senate Republicans press ahead with plans for a Friday test vote.\n\nA top Senate Republican said Thursday the confidential FBI report on charges that Brett Kavanaugh sexually abused women three decades ago "found no hint of misconduct" by the Supreme Court nominee.\n\nSenate Judiciary Committee Chairman Chuck Grassley, R-Iowa, made his remarks — and urged his colleagues to confirm the conservative judge — in a written statement hours after the post-midnight delivery of the FBI document to Congress. With Kavanaugh\'s uncertain prospects for approval depending in part on the decisions of five wavering senators, lawmakers began viewing the document in a secure room in the Capitol complex.\n\n"There\'s nothing in it that we didn\'t already know," Grassley contended, basing his comment on a briefing he said he\'d received from committee aides. He added, "This investigation found no hint of misconduct."\n\nPerhaps more importantly, two key senators, Susan Collins of Maine and Jeff Flake of Arizona, expressed satisfaction with the FBI report.\n\nCollins, who has not indicated how she would vote, said that “It appears to be a very thorough investigation but I’m going back later to personally read the interviews.”\n\nFlake told reporters, “We’ve seen no additional corroborating information.”\n\nCollins, Flake and Sen. Lisa Murkowski, R-Alaska, are the critical votes that could ensure Brett M. Kavanaugh’s nomination to the Supreme Court.\n\nDemocrats have complained that the FBI\'s reopening of its Kavanaugh background check has been far too limited, leaving out contact with crucial potential witnesses. They say some people had reached out to be interviewed by the FBI or the Judiciary Committee but were not questioned.\n\nWhite House spokesman Raj Shah rebuffed that idea, saying, "What critics want is a never-ending fishing expedition into high school drinking." He said the FBI reached out to 10 people and interviewed nine, including "several individuals at the request of the Senate, and had a series of follow-up interviews ... following certain leads."\n\nWhile the FBI interviews were to focus on sexual assault allegations, Democrats have also questioned his drinking habits during high school and college and dishonest comments they say he\'s made about his background. Kavanaugh has said stories of bad behavior while drinking are exaggerated.\n\nGrassley said the FBI could not "locate any third parties who can attest to any of the allegations," and he said there is "no contemporaneous evidence." He provided no specific detail.\n\nSenate Majority Leader Mitch McConnell, R-Ky., has already started a process that will produce a crucial test vote in his polarized chamber Friday on Kavanaugh\'s fate. Should Republicans get the majority of votes they need — and Vice President Mike Pence is available to cast the tie-breaker, if necessary — that would set up a decisive roll call on his confirmation, likely over the weekend.\n\n"Senators ought to wipe away the muck from all the mudslinging and politics and look at this nomination with clear eyes," Grassley said, echoing accusations against Democrats that McConnell has been making. He added, "It\'s time to vote. I\'ll be voting to confirm Judge Kavanaugh."\n\nThree women have accused Kavanaugh of sexual misconduct in separate incidents in the 1980s. Kavanaugh, 53, now a judge on the powerful District of Columbia Circuit Court of Appeals, has denied the claims.\n\nThe White House received the FBI report around 3 a.m. Thursday.\n\nPresident Donald Trump weighed in hours later in a tweet in which he denounced what he called "the harsh and unfair treatment" of Kavanaugh. "This great life cannot be ruined by mean" and "despicable Democrats and totally uncorroborated allegations!"\n\nDemocratic Sen. Tammy Duckworth of Illinois told reporters Thursday that time slots for reading the FBI file are so full that senators are being told they might have to wait until Friday to read it. "They\'re so swamped," she said.\n\nThe report arrived at a Capitol palpably tense over the political stakes of the nomination fight and from aggressive anti-Kavanaugh protesters who have rattled and reportedly harassed senators. Feeding the anxiety was an unusually beefy presence of the U.S. Capitol Police, who were keeping demonstrators and frequently reporters at arm\'s length by forming wedges around lawmakers walking through corridors.\n\nBarring leaks, it was unclear how much of the FBI report, if any, would be made public. While senators from both sides have expressed support for revealing at least parts of the findings, FBI background checks on nominees are supposed to remain confidential.\n\nWith Republicans clinging to a razor-thin 51-49 Senate majority and five senators — including three Republicans — still publicly undeclared, the conservative jurist\'s prospects of Senate confirmation could hinge largely on the file\'s contents.\n\nThe three undecided GOP senators rebuked Trump for mocking one accuser, Christine Blasey Ford, by mimicking her responses to questions at last week\'s dramatic Senate Judiciary Committee hearing.\n\nSen. Susan Collins, R-Maine, told reporters that Trump\'s lampooning of Ford at a Tuesday night Mississippi campaign rally was "just plain wrong." Sen. Lisa Murkowski, R-Alaska, called it "wholly inappropriate and in my view unacceptable," and Sen. Jeff Flake, R-Ariz., said on NBC\'s "Today" show that the remarks were "kind of appalling."\n\nThose senators, along with Democrats Heidi Heitkamp of North Dakota and Joe Manchin of West Virginia, have yet to declare how they will vote.\n\nTrump drew laughs Tuesday with his rendition of how Ford answered questions at last week\'s hearing. "I had one beer — that\'s the only thing I remember," he stated inaccurately.\n\nSenate Minority Leader Chuck Schumer, D-N.Y., said Trump\'s insults marked a "new low."\n\nUnderscoring rising tensions, Democrats suggested that previous FBI background checks of Kavanaugh may have unearthed misconduct by the nominee.\n\nDemocrats wrote to Grassley challenging a Tuesday tweet by GOP aides saying prior investigations never found "a whiff of ANY issue — at all — related in any way to inappropriate sexual behavior or alcohol abuse." Democrats wrote that the GOP tweet contained information that is "not accurate."\n\nCommittee Republicans tweeted in response that their prior tweet was "completely truthful" and accused Democrats of "false smears."\n\nThe California psychology professor has testified that a drunken Kavanaugh sexually abused her in a locked room at a high school party in the 1980s and has said she believed he was trying to rape her.\n\nThe FBI interviewed several people, including three who Ford has said attended a 1982 high school gathering in suburban Maryland where she says Kavanaugh\'s attack occurred, plus another Kavanaugh friend. The agency has also spoken to a second woman, Deborah Ramirez, who has claimed Kavanaugh exposed himself to her at a Yale party when both were freshmen.\n\nThe Associated Press contributed to this report.'

In [73]:
zzz = collect_sources(ex4)

In [74]:
zzz

,source,type,indicator,quoutes,sentence
0,Judiciary Committee,ORG,told,0,"""Judge Kavanaugh should be confirmed on Saturd..."
1,he,,added,1,"""Hopefully, we're 48 hours away from having a ..."
2,Hatch,PERSON,said,1,"""I apologize to him for the way he's been trea..."
3,Hatch,PERSON,said,1,"""""We have found nothing, nothing to corroborat..."
4,she,,said,0,"As they talked to reporters, a Democratic sena..."
5,Senate Republican,NORP,said,0,A top Senate Republican said Thursday the conf...
6,briefing he,,said,0,"""There's nothing in it that we didn't already ..."
7,He,,added,0,"He added, ""This investigation found no hint o..."
8,Flake,,told,1,"”Flake told reporters, “We’ve seen no addition..."
9,He,,said,1,""" He said the FBI reached out to 10 people and..."


# 

In [10]:
z = collect_sources(example)

In [11]:
len(z)

12

In [12]:
z

[[['blasted', '', 'VERB'],
  ['Mazie', 'PERSON', 'PROPN'],
  ['Hirono', 'PERSON', 'PROPN'],
  ['""', 'quotes', ''],
  [[['Democratic', 'NORP', 'ADJ'],
    ['Sen.', '', 'PROPN'],
    ['Mazie', 'PERSON', 'PROPN'],
    ['Hirono', 'PERSON', 'PROPN'],
    ['blasted', '', 'VERB'],
    ['“', '', 'PROPN'],
    ['the', '', 'DET'],
    ['men', '', 'NOUN'],
    ['in', '', 'ADP'],
    ['this', '', 'DET'],
    ['country', '', 'NOUN'],
    [',', '', 'PUNCT'],
    ['”', '', 'PUNCT'],
    ['blaming', '', 'VERB'],
    ['them', '', 'PRON'],
    ['for', '', 'ADP'],
    ['sexual', '', 'ADJ'],
    ['assault', '', 'NOUN'],
    ['and', '', 'CCONJ'],
    ['urged', '', 'VERB'],
    ['them', '', 'PRON'],
    ['to', '', 'ADP'],
    ['“', '', 'PROPN'],
    ['just', '', 'ADV'],
    ['shut', '', 'VERB'],
    ['up', '', 'PART'],
    ['and', '', 'CCONJ'],
    ['step', '', 'VERB'],
    ['up', '', 'PART'],
    ['”', '', 'PUNCT'],
    ['amid', '', 'ADP'],
    ['allegations', '', 'NOUN'],
    ['of', '', 'ADP'],
    ['mis

In [28]:
z1 = collect_sources(ex2)

In [29]:
len(z1)

19

In [30]:
z1

[[['added', '', 'VERB'], ['security', '', 'NOUN'], ['response', '', 'NOUN']],
 [['claimed', '', 'VERB'],
  ['said', '', 'VERB'],
  ['nomination', '', 'NOUN'],
  ['""', 'quotes', '']],
 [['told', '', 'VERB'],
  ['Collins', 'PERSON', 'PROPN'],
  ['R', 'ORG', 'PROPN'],
  ['-', 'ORG', 'PUNCT'],
  ['Maine', 'ORG', 'PROPN'],
  ['""', 'quotes', '']],
 [['called', '', 'VERB'],
  ['Murkowski', 'PERSON', 'PROPN'],
  ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['""', 'quotes', '']],
 [['conceded', '', 'VERB'], ['Republicans', 'NORP', 'PROPN']],
 [['said', '', 'VERB'], ['focus', '', 'NOUN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['Graham', 'PERSON', 'PROPN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['Schumer', 'PERSON', 'PROPN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'],
  ['party', '', 'NOUN'],
  ['1980s', 'DATE', 'NOUN'],
  ['""', 'quotes', '']],
 [['argued', '', 'VERB'],
  ['Democrats', 'NORP', 'PROPN'],
  ['said', '', 'VERB'],
  ['others', '', 'NOUN'],
  ['Kavanaugh', '

In [8]:
doc = nlp(example)
print([(X.text, X.label_) for X in doc.ents])
y = [(X.text, X.label_) for X in doc.ents]
hol = set()
for i in y:
    hol.add(i[1])

[('Democratic', 'NORP'), ('Mazie Hirono', 'PERSON'), ('Trump’s Supreme Court', 'ORG'), ('Brett Kavanaugh', 'PERSON'), ('Tuesday', 'DATE'), ('evening', 'TIME'), ('Hawaii', 'GPE'), ('Democrat', 'NORP'), ('U.S.', 'GPE'), ('FBI', 'ORG'), ('Hirono', 'ORG'), ('FBI', 'ORG'), ('Kavanaugh’s', 'ORG'), ('Christine Blasey Ford', 'PERSON'), ('KAVANAUGH', 'ORG'), ('Hirono', 'ORG'), ('Ford', 'PERSON'), ('Hirono', 'ORG'), ('Ford', 'PERSON'), ('Ford', 'ORG'), ('Washington Post', 'ORG'), ('the weekend', 'DATE'), ('Dianne Feinstein', 'PERSON'), ('D-Calif', 'NORP'), ('California', 'GPE'), ('Kavanaugh', 'ORG'), ('36 years ago', 'DATE'), ('Ford', 'ORG'), ('Kavanaugh', 'PERSON'), ('Mark Judge', 'PERSON'), ('Kavanaugh', 'PERSON'), ('Kavanaugh', 'PERSON'), ('Monday', 'DATE'), ('Sunday', 'DATE'), ('KAVANAUGH', 'ORG'), ('The Senate Judiciary Committee', 'ORG'), ('Ford', 'ORG'), ('Kavanaugh', 'ORG'), ('Kavanaugh', 'PERSON'), ('Ford', 'ORG'), ('Monday', 'DATE'), ('Ford', 'ORG'), ('FBI', 'ORG'), ('Fox News', 'ORG')

In [10]:
hol

{'DATE', 'GPE', 'NORP', 'ORG', 'PERSON', 'TIME'}

In [5]:
# Token and Tag 
for token in doc: 
    print(token, token.pos_, token.ent_type_) 
# token.tag_, token.dep_,
# You want list of Verb tokens 
#print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"]) 

Democratic ADJ NORP
Sen. PROPN 
Mazie PROPN PERSON
Hirono PROPN PERSON
blasted VERB 
“ PROPN 
the DET 
men NOUN 
in ADP 
this DET 
country NOUN 
, PUNCT 
” PUNCT 
blaming VERB 
them PRON 
for ADP 
sexual ADJ 
assault NOUN 
and CCONJ 
urged VERB 
them PRON 
to ADP 
“ PROPN 
just ADV 
shut VERB 
up PART 
and CCONJ 
step VERB 
up PART 
” PUNCT 
amid ADP 
allegations NOUN 
of ADP 
misconduct NOUN 
against ADP 
President PROPN 
Trump PROPN ORG
’s PROPN ORG
Supreme PROPN ORG
Court PROPN ORG
nominee NOUN 
Judge PROPN 
Brett PROPN PERSON
Kavanaugh PROPN PERSON
. PUNCT 


 SPACE 
At ADP 
a DET 
press NOUN 
conference NOUN 
Tuesday PROPN DATE
evening NOUN TIME
, PUNCT 
the DET 
Hawaii PROPN GPE
Democrat PROPN NORP
made VERB 
the DET 
sweeping ADJ 
generalization NOUN 
against ADP 
“ PUNCT 
the DET 
men NOUN 
” PUNCT 
in ADP 
the DET 
U.S. PROPN GPE
for ADP 
“ PROPN 
perpetuating VERB 
” PUNCT 
sexual ADJ 
assault NOUN 
. PUNCT 


 SPACE 
“ PROPN 
I PRON 
expect VERB 
the DET 
men NOUN 
in ADP 
t

In [9]:
# define indicator words
indicators = ['said', 'added', 'told', 'made', 'blasted', 'suggested', 
                         'claimed', 'called', 'conceded', 'argued']


def collect_sources(article):
    doc = nlp(article)
    
    
    
    sentences = []
    sources = []
    sent = []
    for token in doc: 
        word = str(token)
        entity = str(token.ent_type_)
        pos = str(token.pos_)
        pair = [word, entity, pos]
        sent.append(pair)
        # construct sentences
        punc = ['.', '?', '!']
        if word in punc:
            sentences.append(sent)
            sent = []
    # loop through sentences constructing sources
    collection = []
    for s in sentences:
        co = []
        sources = []
        quote_bool = 0
        for idx, val in enumerate(s):
            if val[0] in indicators:
                co.append(val)
                for j in s[max(idx-6, 0):idx]:
                    if j[1] in {'NORP', 'ORG', 'PERSON'} and j[2]=='PROPN':
                        co.append(j)
#                     if j[2] in {'NOUN'}:
#                         co.append(j)
            quotes = ['“' , '”' , '"']
            if val[0] in quotes:
                quote_bool = 1
        if quote_bool:
            co.append(['""', 'quotes', ''])
        co.append([s, 'sentence', ''])
        if len(co)>=3:
            collection.append(co)     
    return collection



# doc = nlp(example)
# print([(X.text, X.label_) for X in doc.ents])
# y = [(X.text, X.label_) for X in doc.ents]
# hol = set()
# for i in y:
#     hol.add(i[1])